# ANU ASTR4004 2025 - Week 8

Author: Dr Sven Buder (sven.buder@anu.edu.au)

based on Viviana Acquaviva's notebooks and the scikit learn tutorial:  
https://ogrisel.github.io/scikit-learn.org/sklearn-tutorial/tutorial/astronomy/dimensionality_reduction.html#sdss-spectral-data


In [ ]:
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import patches
from sklearn import preprocessing, decomposition

# Make the size and fonts larger for this presentation
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 12

## Dimensionality reduction of galaxy spectra

### The data: 4000 spectra, 1000 features

In [ ]:
data = np.load('data/spec4000_corrected.npz')

In [ ]:
wavelengths = data['wavelengths']
X = data['X']
y = data['y']
labels = data['labels'].astype('str')

In [ ]:
X.shape

In [ ]:
y

In [ ]:
# Let's plot a few spectra
plt.figure(figsize=(10,5))

for i_class in (2, 3, 4, 5, 6):
    i = np.where(y == i_class)[0][0]
    l = plt.plot(wavelengths, X[i] + 20 * i_class)
    c = l[0].get_color()
    plt.text(6800, 2 + 20 * i_class, labels[i_class], color=c)

plt.subplots_adjust(hspace=0)
plt.xlabel('wavelength (Angstroms)')
plt.ylabel('flux + offset')
plt.title('Sample of Spectra');
plt.show()

### Let's throw a PCA onto it

In [ ]:
#  Perform PCA

scaler = preprocessing.StandardScaler() #It's important that data are centered!

Xn = scaler.fit_transform(X) #This is a standardization procedure.

pca_1 = decomposition.PCA(n_components=1, random_state=0)

pca_2 = decomposition.PCA(n_components=2, random_state=0)

pca_5 = decomposition.PCA(n_components=5, random_state=0)

pca_50 = decomposition.PCA(n_components=50, random_state=0)

pca_100 = decomposition.PCA(n_components=100, random_state=0)

pca_1000 = decomposition.PCA(n_components=1000, random_state=0)

X_proj_1 = pca_1.fit_transform(Xn) #the projected data set - it lives in a new feature space with 4,000 objects and 1 features

X_proj_2 = pca_2.fit_transform(Xn) #the projected data set - it lives in a new feature space with 4,000 objects and 2 features

X_proj_5 = pca_5.fit_transform(Xn) #the projected data set - it lives in a new feature space with 4,000 objects and 5 features

X_proj_50 = pca_50.fit_transform(Xn) #the projected data set - it lives in a new feature space with 4,000 objects and 50 features

X_proj_100 = pca_100.fit_transform(Xn) #the projected data set - it lives in a new feature space with 4,000 objects and 100 features

X_proj_1000 = pca_1000.fit_transform(Xn) #the projected data set - it lives in a new feature space with 4,000 objects and 1000 features

In [ ]:
plt.figure()

l = plt.plot(wavelengths, pca_50.mean_ - 0.15)
c = l[0].get_color()
plt.text(7000, -0.16, "mean", color=c) 

# In linear PCA, the first eigenvector is always the mean, 
# and the first n components are always the same

for i in range(4):
    
    l = plt.plot(wavelengths, pca_50.components_[i] + 0.15 * i)
    
    l = plt.plot(wavelengths, pca_100.components_[i] + 0.15 * i, linestyle = '-.')
    
    c = l[0].get_color()
    
    plt.text(7000, -0.01 + 0.15 * i, "component %i" % (i + 1), color=c)

    plt.ylim(-0.2, 0.6)
    
plt.xlabel('wavelength (Angstroms)')
plt.ylabel('scaled flux + offset')
plt.title('Mean Spectrum and Eigen-spectra')

plt.show()

In [ ]:
f, gs = plt.subplots(1,2,figsize=(10,4))

gs[0].plot(np.arange(50)+1,pca_50.explained_variance_ratio_)
gs[0].set_ylabel('Variation explained \n by each component')
gs[0].set_xlabel('Component')

gs[1].plot(np.arange(50)+1,np.cumsum(pca_50.explained_variance_ratio_))
gs[1].set_ylabel('Variation explained \n by all components')
gs[1].set_xlabel('Component')
gs[1].set_xlim(0,10)

plt.tight_layout()
plt.show()

In [ ]:
# Reconstruct spectra
print(np.shape(Xn))
X_recon_1 = pca_1.inverse_transform(X_proj_1)
X_recon_2 = pca_2.inverse_transform(X_proj_2)
X_recon_5 = pca_5.inverse_transform(X_proj_5)
X_recon_50 = pca_50.inverse_transform(X_proj_50)
X_recon_100 = pca_100.inverse_transform(X_proj_100)
X_recon_1000 = pca_1000.inverse_transform(X_proj_1000)

# Pick 5 random stars
np.random.seed(902)
n_examples = 5
idx = np.random.choice(X.shape[0], size=n_examples, replace=False)

wavelengths = range(X.shape[1])  # or real wavelength array if you have one

fig, axes = plt.subplots(n_examples, 6, figsize=(16, 10), sharex=True, sharey=True)

for row, i in enumerate(idx):
    # Original
    axes[row, 0].plot(wavelengths, Xn[i], color="black")
    axes[row, 0].set_title("Original (normalized)")
    
    # Reconstructed with 1 PCs
    axes[row, 1].plot(wavelengths, X_recon_1[i], color="C0")
    axes[row, 1].set_title("PCA 1")

    # Reconstructed with 2 PCs
    axes[row, 2].plot(wavelengths, X_recon_2[i], color="C1")
    axes[row, 2].set_title("PCA 2")

    # Reconstructed with 50 PCs
    axes[row, 3].plot(wavelengths, X_recon_5[i], color="C2")
    axes[row, 3].set_title("PCA 5")

    # Reconstructed with 50 PCs
    axes[row, 4].plot(wavelengths, X_recon_50[i], color="C3")
    axes[row, 4].set_title("PCA 50")
    
    # Reconstructed with 1000 PCs
    axes[row, 5].plot(wavelengths, X_recon_1000[i], color="C4")
    axes[row, 5].set_title("PCA 1000")

for ax in axes[:, 0]:
    ax.set_ylabel("Flux (scaled)")
for ax in axes[-1, :]:
    ax.set_xlabel("Pixels")

plt.tight_layout()
plt.show()
